# Homework 10

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ISLP.models import ModelSpec as MS

In [5]:
Hitters = pd.read_csv('C:\\Users\\lorentsen\\Documents\\Python\\Math-180-Applied-Statistics-and-Machine-Learning\\ISLRv2_data\\ALL CSV FILES - 2nd Edition\\Hitters.csv')
Hitters.dropna(inplace=True)
y = Hitters['Salary']
x = Hitters.drop('Salary', axis=1)

## Problem 1
### part a)

In [6]:
x = x.astype({'AtBat': 'float64', 'Hits': 'float64', 'HmRun': 'float64', 'Runs': 'float64', 'RBI': 'float64', 'Walks': 'float64', 'Years': 'float64', 'CAtBat': 'float64', 'CHits': 'float64', 'CHmRun': 'float64', 'CRuns': 'float64', 'CRBI': 'float64', 'CWalks': 'float64', 'PutOuts': 'float64', 'Assists': 'float64', 'Errors': 'float64'})
for i in range(0, x.shape[1]):
    if x.iloc[:, i].dtype == 'object':
        continue
    x.iloc[:,i] = (x.iloc[:, i] - x.iloc[:, i].mean())/x.iloc[:, i].std()

y = (y - y.mean())/y.std()
x['NewLeague'] = x['NewLeague'].map({'A': 1, 'N': 0})
x['League'] = x['League'].map({'A': 1, 'N': 0})
x['Division'] = x['Division'].map({'E': 1, 'W': 0})
# print(x.head())


### part b)

In [7]:
import statsmodels.api as sm

In [8]:
# newmodel = MS(x, intercept = False)
# X = newmodel.fit_transform(x)
# Y = y
model = sm.OLS(y, x).fit()
model.params

AtBat       -0.658632
Hits         0.755723
HmRun        0.080603
Runs        -0.121588
RBI         -0.047869
Walks        0.291328
Years       -0.021399
CAtBat      -0.892974
CHits        0.138303
CHmRun      -0.033312
CRuns        1.132763
CRBI         0.581068
CWalks      -0.475808
League      -0.168323
Division     0.206751
PutOuts      0.174039
Assists      0.122905
Errors      -0.052294
NewLeague    0.020377
dtype: float64

### part c)

We have
\begin{equation}
f(X) = \beta_0 + \sum^K_{k=1} \beta_k h_k(X)
\end{equation}
where $h_k$ is our activation function, which is just connected to one predictor for each $k$. We can rewrite this as
\begin{equation}
f(X) = \beta_0 +\sum^p_{k=1} \beta_k X_k
\end{equation}
Which is just linear regression.

### part d)


In [9]:

newmodel = MS(Hitters.drop('Salary', axis=1), intercept = False)
X = newmodel.fit_transform(Hitters.drop('Salary', axis=1))
Y = Hitters['Salary']

X['NewLeague'] = X['NewLeague'].map({'A': 1, 'N': 0})
X['League'] = X['League'].map({'A': 1, 'N': 0})
X['Division'] = X['Division'].map({'E': 1, 'W': 0})

### part e)

In [10]:
import keras
from keras import layers
from keras import ops

In [11]:
X.shape

(263, 19)

In [12]:
modelnn = keras.Sequential(
    [
        # Input(shape),
        layers.Dense(1, activation = None, input_shape = (X.shape[1],)),
    ]
)

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### part f)

In [13]:
modelnn.compile(loss='mean_squared_error', optimizer='adam')
modelnn.fit(X, Y, epochs=200, verbose=0)

### part g)

In [14]:
showparams = pd.DataFrame()
# print(model.params)
showparams['glm'] = model.params
showparams['nn'] = modelnn.get_weights()[0]
# showparams.index = model.params.index
print(showparams)

                glm        nn
AtBat     -0.658632  0.372490
Hits       0.755723  0.220966
HmRun      0.080603  0.427546
Runs      -0.121588  0.152816
RBI       -0.047869  0.203717
Walks      0.291328  0.282414
Years     -0.021399  0.345568
CAtBat    -0.892974 -0.036327
CHits      0.138303  0.225783
CHmRun    -0.033312  0.520898
CRuns      1.132763  0.640141
CRBI       0.581068 -0.247382
CWalks    -0.475808 -0.019566
League    -0.168323  0.475366
Division   0.206751  0.481771
PutOuts    0.174039  0.294817
Assists    0.122905  0.077672
Errors    -0.052294  0.237375
NewLeague  0.020377 -0.279441


Seems like it does not match the glm very well

In [15]:
dir(keras.optimizers)

['Adadelta',
 'Adafactor',
 'Adagrad',
 'Adam',
 'AdamW',
 'Adamax',
 'Ftrl',
 'Lamb',
 'Lion',
 'LossScaleOptimizer',
 'Nadam',
 'Optimizer',
 'RMSprop',
 'SGD',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'deserialize',
 'get',
 'legacy',
 'schedules',
 'serialize']

In [16]:
modelnn.compile(loss='mean_squared_error', optimizer = 'Adagrad')
modelnn.fit(X, Y, epochs=200, verbose=0)
showparams['nn_adagrad'] = modelnn.get_weights()[0]

modelnn.compile(loss='mean_squared_error', optimizer = 'Adamax')
modelnn.fit(X, Y, epochs=200, verbose=0)
showparams['nn_Adamax'] = modelnn.get_weights()[0]


In [17]:
print(showparams)

                glm        nn  nn_adagrad  nn_Adamax
AtBat     -0.658632  0.372490    0.370088   0.361001
Hits       0.755723  0.220966    0.228304   0.265076
HmRun      0.080603  0.427546    0.438068   0.479720
Runs      -0.121588  0.152816    0.161307   0.202442
RBI       -0.047869  0.203717    0.214516   0.262501
Walks      0.291328  0.282414    0.297790   0.381039
Years     -0.021399  0.345568    0.332632   0.295523
CAtBat    -0.892974 -0.036327   -0.042506  -0.051617
CHits      0.138303  0.225783    0.223206   0.229322
CHmRun    -0.033312  0.520898    0.533551   0.590620
CRuns      1.132763  0.640141    0.638909   0.648556
CRBI       0.581068 -0.247382   -0.240997  -0.202009
CWalks    -0.475808 -0.019566   -0.024043  -0.030344
League    -0.168323  0.475366    0.453000   0.350029
Division   0.206751  0.481771    0.520817   0.690721
PutOuts    0.174039  0.294817    0.298553   0.313319
Assists    0.122905  0.077672    0.063768   0.019146
Errors    -0.052294  0.237375    0.215570   0.

They only vary a little betwwen different optimizers, but still far off the glm, which is weird.

## Problem 2

#### part a)

In [18]:
import statsmodels.api as sm

In [19]:
default = pd.read_csv('C:\\Users\\lorentsen\\Documents\\Python\\Math-180-Applied-Statistics-and-Machine-Learning\\ISLRv2_data\\ALL CSV FILES - 2nd Edition\\Default.csv')

default.head()
default['default'] = default['default'].map({'No': 0, 'Yes': 1})
default['student'] = default['student'].map({'No': 0, 'Yes': 1})

X = MS(default.columns.drop('default'), intercept = False).fit_transform(default)
Y = default['default']

GLMmodel = sm.GLM(
    Y
    , X
    , family = sm.families.Binomial()
    )
regr = GLMmodel.fit()
regr.params

student   -3.753126
balance    0.002795
income    -0.000174
dtype: float64

#### part b) 
See pdf
#### part c)


In [20]:
modelnn = keras.Sequential(
    [
        layers.Dense(2, activation = 'softmax', input_shape = (X.shape[1],))
    ]
)

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


#### part e)

In [21]:
modelnn.compile(loss='categorical_crossentropy', optimizer = 'adam')
modelnn.fit(X, keras.utils.to_categorical(Y), epochs=200, verbose=0)

<!-- #### part f) -->

In [22]:
compareparams = pd.DataFrame()
compareparams['glm'] = regr.params
compareparams['nn'] = np.diff(modelnn.get_weights()[0], axis = 1)
compareparams

,glm,nn
student,-3.753126,-11.298038
balance,0.002795,0.015404
income,-0.000174,-0.000915


They are fairly similar

In [29]:
##Convert to HTML
!jupyter nbconvert --to html HW10.ipynb



[NbConvertApp] Converting notebook HW10.ipynb to html
[NbConvertApp] Writing 318916 bytes to HW10.html
